In [ ]:
%load_ext autoreload
%autoreload 2
import logging
logging.basicConfig(level=logging.INFO)
import matplotlib.pyplot as plt

import geopandas as gpd
from eodag import EODataAccessGateway
import rioxarray as riox
import numpy as np
import folium
from folium import LayerControl
from eo_tools_dev.util import show_cog
from eo_tools.S1.process import goldstein

# credentials need to be stored in the following file (see EODAG docs)
confpath = "/data/eodag_config.yml"
dag = EODataAccessGateway(user_conf_file_path=confpath)
# make sure cop_dataspace will be used 
dag.set_preferred_provider("cop_dataspace")
log = logging.getLogger(__name__)

In [ ]:
# change to your custom locations
data_dir = "/data/S1"

ids = [
 "S1A_IW_SLC__1SDV_20230904T063730_20230904T063757_050174_0609E3_DAA1", 
 "S1A_IW_SLC__1SDV_20230916T063730_20230916T063757_050349_060FCD_6814"
]
primary_dir = f"{data_dir}/{ids[0]}.SAFE"
secondary_dir = f"{data_dir}/{ids[1]}.SAFE"
outputs_prefix="/data/res/dev-goldstein"

In [ ]:
# load a geometry
file_aoi = "../data/Morocco_small.geojson"
# file_aoi = "../data/Morocco_AOI.geojson"
shp = gpd.read_file(file_aoi).geometry[0]

## Pre-process InSAR pair

In [ ]:
from eo_tools.S1.process import prepare_insar

# out_dir = prepare_insar(
#     dir_prm=primary_dir,
#     dir_sec=secondary_dir,
#     outputs_prefix=outputs_prefix,
#     aoi_name=None,
#     shp=shp,
#     pol="full",
#     subswaths=["IW1", "IW2", "IW3"],
#     cal_type="sigma",
#     apply_fast_esd=False,
#     dem_upsampling=1.8,
#     dem_force_download=False,
#     dem_buffer_arc_sec=40,
#     warp_kernel="bicubic",
# )

In [ ]:
from pathlib import Path
out_dir = list(Path.glob(Path(outputs_prefix), "S1*"))[0] / "sar"

## Goldstein 

In [ ]:
import rasterio
test_file = "/data/test_file.tif"
test_arr = np.random.rand(4096, 4096) + 1j * np.random.rand(4096, 4096) 
prof = dict(width=4096, height=4096, count=1, dtype="complex64")
with rasterio.open(test_file, 'w', **prof) as ds:
    ds.write(test_arr, 1)

In [ ]:
test_file = "/data/test_file.tif"
test_out = "/data/test_out.tif"
goldstein(test_file, test_out)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 10))
# plt.imshow(ifg.real, interpolation="none")
# plt.imshow(arrout.real, interpolation="none")
plt.colorbar(fraction=0.046, pad=0.04)

In [ ]:
np.random.rand(1)

## Apply processing chains: coherence and change detection

In [ ]:
from eo_tools.S1.process import interferogram
from eo_tools.S1.process import apply_to_patterns_for_pair, apply_to_patterns_for_single
from pathlib import Path

geo_dir = Path(out_dir).parent

# compute interferograms
apply_to_patterns_for_pair(
    interferogram,
    out_dir=out_dir,
    file_prm_prefix="slc_prm",
    file_sec_prefix="slc_sec",
    file_out_prefix="ifg0",
    multilook=[1,4]
)

# apply_to_patterns_for_single(
#     goldstein,
#     out_dir=out_dir,
#     file_in_prefix="ifg0",
#     file_out_prefix="ifg",
#     alpha=0.5,
#     overlap=14,
# )

## Apply geocoding, merge and crop subswaths

In [ ]:
from eo_tools.S1.process import geocode_and_merge_iw
geo_dir = Path(out_dir).parent
geocode_and_merge_iw(geo_dir, shp=shp, var_names=["ifg0", "ifg"], clip_to_shape=False)

## Visualize
Click on top right icon to hide/show layers

In [ ]:
from eo_tools_dev.util import palette_phi
m = folium.Map()
_ = show_cog(f"{geo_dir}/phi_vv.tif", m, rescale="-3.14,3.14", colormap=palette_phi())
_ = show_cog(f"{geo_dir}/phi0_vv.tif", m, rescale="-3.14,3.14", colormap=palette_phi())
LayerControl().add_to(m)
m

In [ ]:
file_ifg_sar = out_dir / "ifg0_vv_iw1.tif"
ifg_sar = riox.open_rasterio(file_ifg_sar)
plt.figure(figsize=(10, 10))
plt.imshow(np.angle(ifg_sar[0]), interpolation="none")
plt.colorbar(fraction=0.046, pad=0.04)


In [ ]:
file_ifg_sar = out_dir / "ifg_vv_iw1.tif"
ifg_sar_filt = riox.open_rasterio(file_ifg_sar)
plt.figure(figsize=(10, 10))
plt.imshow(np.angle(ifg_sar_filt[0]), interpolation="none")
plt.colorbar(fraction=0.046, pad=0.04)


In [ ]:
import matplotlib.pyplot as plt

# sl1 = np.s_[2750:3250, 500:1500]
# sl2 = np.s_[2000:3000, 4000:5000]
sl3 = np.s_[1500:1564, 500:564]
arr = ifg_sar[0][sl3].data

# alpha = 0.5
alpha = 0.9001
smooth = lambda x: uflt(x, 3) #+ 1j*uflt(x, 3)
Z = fftshift(fft2(arr))
H = smooth(abs(Z))**(alpha)
arrout = ifft2(ifftshift(H*Z))

plt.figure(figsize=(10, 10))
plt.imshow(H, interpolation="none")
plt.colorbar(fraction=0.046, pad=0.04)